# Exercise 4

Work on this before the next lecture on 1 May. We will talk about questions, comments, and solutions during the exercise after the third lecture.

Please do form study groups! When you do, make sure you can explain everything in your own words, do not simply copy&paste from others.

The solutions to a lot of these problems can probably be found with Google. Please don't. You will not learn a lot by copy&pasting from the internet.

If you want to get credit/examination on this course please upload your work to **your GitHub repository** for this course **before** the next lecture starts and post a link to your repository [in this thread](). If you worked on things together with others please add their names to the notebook so we can see who formed groups.

---

These are some useful default imports for plotting and [`numpy`](http://www.numpy.org/)

In [17]:
%config InlineBackend.figure_format='retina'
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14
from sklearn.utils import check_random_state

## Pitfalls of estimating model performance

This question sets up a classification problem to illustrate a common pitfall in
evaluating model performance. To keep things simple the `y`s in this classroom problem
are picked at random: there is no way for a classifier to learn how to model `y` based
on the features provided. This means we know what the true accuracy is: 0.5.

In [18]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_regression

np.random.seed(6450345)

A common task when building a new model is to select only those variables that are "best"
for the problem. This selection procedure can take many different shapes, here we will
compute the correlation of each feature with the target, select the 20 features that
have the highest correlation and use those in our gradient boosted tree ensemble.

We will then use cross validation to evaluate the performance.

In [19]:
def make_data(N=1000, n_vars=10,
              n_classes=2):
    X = np.random.normal(size=(N,n_vars))
    y = np.random.choice(n_classes, N)
    
    return X, y

X,y = make_data(N=2000, n_vars=50000)

select = SelectKBest(f_regression, k=20)
X_sel = select.fit_transform(X, y)

clf = GradientBoostingClassifier()
scores = cross_val_score(clf, X_sel, y, cv=5, n_jobs=8)

print("Scores on each subset:")
print(scores)
avg = (100*np.mean(scores), 100*np.std(scores)/np.sqrt(scores.shape[0]))
print("Average score and uncertainty: (%.2f +- %.3f)%%" % avg)

Scores on each subset:
[ 0.60349127  0.63092269  0.5975      0.63157895  0.65413534]
Average score and uncertainty: (62.35 +- 0.924)%


What just happened? We have a classifier that achieves an accuracy of ~60% but we know that the features are uncorrelated with the target. How did this happen? What mistake did we make?

What do we need to repair this? How do we know it is repaired? When the predicted performance
is close to what we know to be the true performance?

The error is that to select the important features we used the whole data. And to evaluate the performance we didn't use the part of the data that was locked in a vault, but the same data we used to train our model. So to fix this, we are going to split our data into train and test set first, and then train on the train set and use test set only for the evaluation.

In [30]:
# your answer here
from sklearn.model_selection import train_test_split

X, y = make_data(N=2000, n_vars=50000)
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, train_size=0.7)

#use only train set to select the features
select = SelectKBest(f_regression, k=20)
select.fit(X_train, y_train)
X_train = select.transform(X_train)
# we transform X_test, but we don't use it till the final verification
X_test = select.transform(X_test)

clf = GradientBoostingClassifier(random_state=42)
scores = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=8)

print("Scores on each subset:")
print(scores)
avg = (100*np.mean(scores), 100*np.std(scores)/np.sqrt(scores.shape[0]))
print("Average score and uncertainty: (%.2f +- %.3f)%%" % avg)

clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print("\nScore that we should actually use to estimate the performance:")
print("%.2f" % accuracy)

Scores on each subset:
[ 0.64412811  0.60142349  0.56785714  0.67383513  0.68100358]
Average score and uncertainty: (63.36 +- 1.932)%

Score that we should actually use to estimate the performance:
0.50


Now we see that the score is indeed around 0.5. We can actually loop over the random_state to make sure, this is not a random down fluctuation.

In [31]:
random_states = np.arange(0, 100)
clf = GradientBoostingClassifier()
accuracies = []
for random_state in random_states:
    clf = GradientBoostingClassifier()
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    accuracies.append(accuracy)
    
score_mean = np.mean(accuracies)
score_std = np.std(accuracies)
print("Final score %.2f +- %.3f" % (score_mean, score_std))

Final score 0.50 +- 0.001
